In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import MobileNetV2
from keras.layers import AveragePooling2D, Dropout, Flatten, Dense, Input
from keras.models import Model, Sequential
from keras.optimizers import Adam
from keras.applications.mobilenet_v2 import preprocess_input
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import numpy as np
import pickle
import os
import pandas as pd
import GetPickles
import var
import Viz

In [ ]:
def get_mobile_net(dim):
    model = Sequential()
    baseModel = MobileNetV2(weights="imagenet", include_top=False,
        input_tensor=Input(shape=dim))
    
    model.add(baseModel)
    headModel = model.add(AveragePooling2D(pool_size=(7, 7)))
    headModel = model.add(Flatten(name="flatten"))
    headModel = model.add(Dense(256, activation="relu"))
    headModel = model.add(Dropout(0.3))
    headModel = model.add(Dense(3, activation="softmax", name = 'Output'))

    # place the head FC model on top of the base model (this will become
    # the actual model we will train)


    # loop over all layers in the base model and freeze them so they will
    # *not* be updated during the first training process
    for layer in baseModel.layers:
        layer.trainable = False
    
    model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    return model

In [ ]:
x_train, x_test, y_train, y_test = GetPickles.get_samples('mobilenet')
print(x_train.shape, x_test.shape) 
print(y_train.shape, y_test.shape)

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator


augment = True 

early_stopping = EarlyStopping(monitor='val_loss', verbose = 1, patience=35)
model_checkpoint = ModelCheckpoint('../models/MobileNet-ModelCheckpointWeights.h5', verbose = 1, save_best_only=True,
                                  monitor = 'val_loss')
lr_plat = ReduceLROnPlateau(patience = 5, mode = 'min')
epochs = 2000
batch_size = 16
if var.img_type == 'grey': 
    dim = (var.mobilenet_dimension[0], var.mobilenet_dimension[1], 1)
else: 
    dim = (var.mobilenet_dimension[0], var.mobilenet_dimension[1], 3)
    
mobile_net = get_mobile_net(dim =dim)

if augment: 
    augmentation =ImageDataGenerator(rotation_range = 15, width_shift_range = .1, height_shift_range = .1, 
                                                           horizontal_flip = True, fill_mode = 'nearest')
    augmentation.fit(x_train)
    mobile_net_history = mobile_net.fit_generator(augmentation.flow(x_train, y_train, batch_size = batch_size),
                epochs = epochs, steps_per_epoch = 8,
         callbacks = [early_stopping, model_checkpoint, lr_plat], validation_data = (x_test, y_test), verbose= 1)
else: 
    
    mobile_net_history = mobile_net.fit(x_train, y_train, batch_size = batch_size,
                epochs = epochs, 
         callbacks = [early_stopping, model_checkpoint, lr_plat], validation_data = (x_test, y_test), verbose= 1)


In [ ]:
print(mobile_net.summary())

### Loss and Accuracy

In [ ]:
Viz.plot_loss_accuracy(mobile_net_history)

### ROC and AUC

In [ ]:
if var.img_type == 'grey': 
    dim = (var.mobilenet_dimension[0], var.mobilenet_dimension[1], 1)
else: 
    dim = (var.mobilenet_dimension[0], var.mobilenet_dimension[1], 3)
    
    
mobilenet = get_mobile_net(dim)
mobilenet.load_weights('../models/MobileNet-ModelCheckpointWeights.h5') #load the best weights before overfitting

Viz.plot_roc_auc(mobilenet, x_test, y_test)

### Confusion Matrix

In [ ]:
if var.img_type == 'grey': 
    dim = (var.mobilenet_dimension[0], var.mobilenet_dimension[1], 1)
else: 
    dim = (var.mobilenet_dimension[0], var.mobilenet_dimension[1], 3)
    
    
mobilenet = get_mobile_net(dim)
mobilenet.load_weights('../models/MobileNet-ModelCheckpointWeights.h5') #load the best weights before overfitting





y_test_prob = [np.argmax(i) for i in mobilenet.predict(x_test)]
y_test_labels = [np.argmax(
    i) for i in y_test]
test_cnf = confusion_matrix(y_test_labels, y_test_prob)

y_train_prob = [np.argmax(i) for i in mobilenet.predict(x_train)]
y_train_labels = [np.argmax(i) for i in y_train]
train_cnf = confusion_matrix(y_train_labels, y_train_prob)

Viz.plot_model_cm(test_cnf, train_cnf, classes = ['No Weapon', 'Handgun', 'Rifle'])


In [ ]:
assert False

### Test on Google

In [ ]:
import cv2
from lime import lime_image
from skimage.segmentation import mark_boundaries

In [ ]:
base_path = '../TestImages'
if var.img_type == 'grey': 
    dim = (var.mobilenet_dimension[0], var.mobilenet_dimension[1], 1)
else: 
    dim = (var.mobilenet_dimension[0], var.mobilenet_dimension[1], 3)
    
    
mobilenet = get_mobile_net(dim)
mobilenet.load_weights('../models/MobileNet-ModelCheckpointWeights.h5') #load the best weights before overfitting

for file in os.listdir(base_path): 
    if file == '.ipynb_checkpoints':
        continue
    full_path = f'{base_path}/{file}'
    img = GetPickles.get_image_value(full_path, var.mobilenet_dimension, var.img_type)
    img = img.reshape(1, img.shape[0], img.shape[1], img.shape[2])
    pred = mobilenet.predict(img)[0]
    print(f'{file}\t\t{np.argmax(pred)}\t\t{pred.max()}\t\t{pred}')